# DNN Critic model

In [1]:
import pandas as pd 
import numpy as np  
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# read data
mydt = pd.read_csv("../dataset/UGR/trainbot.csv")
print(mydt['Label'].value_counts())
# Create matrix of features and matrix of target variable 
dataset = mydt.values
X = dataset[:,0:19]
y = dataset[:,19]

# convert the scaled array to dataframe 
#min_max_scaler = MinMaxScaler()
#X_scale = min_max_scaler.fit_transform(X)

X_full = X
y_full = y

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

# define the keras model
model = Sequential()
model.add(Dense(128, input_shape=(19,), activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Building Critic WGAN model...")
# fit the keras model on the dataset
mod = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_data=(X_test, y_test))

# make class predictions with the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))

%matplotlib inline
# Testing 
cm = confusion_matrix(y_test, y_pred) 
tnr = recall_score(y_test, y_pred) 
fpr = 1 - tnr

print ("confusion_matrix for test data\n",cm)
print(classification_report(y_test,y_pred,labels=np.unique(y_pred)))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('Precision: %.4f' % precision_score(y_test, y_pred))
print('Recall: %.4f' % recall_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred))
print('FPR Score: %.4f' % fpr)

# Save model
# serialize model to JSON
model_json = model.to_json()
with open("DNN_critic_model_UGR_2.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("DNN_critic_model_UGR_2.h5")
print("Saved model!")

0    14029
1     1430
Name: Label, dtype: int64
Counter({0.0: 9791, 1.0: 1030})
Counter({0.0: 4238, 1.0: 400})


2022-09-10 06:02:01.599859: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Building Critic WGAN model...
Epoch 1/100
1083/1083 [==============================] - 7s 5ms/step - loss: 0.1735 - accuracy: 0.9238 - val_loss: 0.1130 - val_accuracy: 0.9467
Epoch 2/100
1083/1083 [==============================] - 5s 4ms/step - loss: 0.1079 - accuracy: 0.9499 - val_loss: 0.0983 - val_accuracy: 0.9595
Epoch 3/100
1083/1083 [==============================] - 5s 5ms/step - loss: 0.0946 - accuracy: 0.9584 - val_loss: 0.0913 - val_accuracy: 0.9584
Epoch 4/100
1083/1083 [==============================] - 5s 4ms/step - loss: 0.0873 - accuracy: 0.9644 - val_loss: 0.0886 - val_accuracy: 0.9651
Epoch 5/100
1083/1083 [==============================] - 5s 5ms/step - loss: 0.0822 - accuracy: 0.9657 - val_loss: 0.0784 - val_accuracy: 0.9700
Epoch 6/100
1083/1083 [==============================] - 5s 4ms/step - loss: 0.0787 - accuracy: 0.9675 - val_loss: 0.0815 - val_accuracy: 0.9681
Epoch 7/100
1083/1083 [==============================] - 5s 4ms/step - loss: 0.0760 - accuracy: 0.96

Epoch 57/100
1083/1083 [==============================] - 4s 4ms/step - loss: 0.0322 - accuracy: 0.9881 - val_loss: 0.0520 - val_accuracy: 0.9858
Epoch 58/100
1083/1083 [==============================] - 3s 3ms/step - loss: 0.0333 - accuracy: 0.9878 - val_loss: 0.0492 - val_accuracy: 0.9838
Epoch 59/100
1083/1083 [==============================] - 4s 3ms/step - loss: 0.0314 - accuracy: 0.9892 - val_loss: 0.0492 - val_accuracy: 0.9845
Epoch 60/100
1083/1083 [==============================] - 4s 3ms/step - loss: 0.0327 - accuracy: 0.9884 - val_loss: 0.0518 - val_accuracy: 0.9851
Epoch 61/100
1083/1083 [==============================] - 4s 3ms/step - loss: 0.0318 - accuracy: 0.9885 - val_loss: 0.0551 - val_accuracy: 0.9830
Epoch 62/100
1083/1083 [==============================] - 3s 3ms/step - loss: 0.0319 - accuracy: 0.9889 - val_loss: 0.0496 - val_accuracy: 0.9860
Epoch 63/100
1083/1083 [==============================] - 3s 3ms/step - loss: 0.0307 - accuracy: 0.9890 - val_loss: 0.0546 -

# WGAN

In [2]:
import torch 
import torch.nn
import torch.nn.functional as nn 
import torch.autograd as autograd 
import torch.optim as optim 
import torch.utils.data as Data
from torch.autograd import Variable
import numpy as np 
import tensorflow as tf 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix 
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers
from keras.models import model_from_json
import matplotlib 
matplotlib.use('agg') 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

# hyperparameters 
mb_size = 64 
z_dim = 10 
X_dim = 19 
y_dim = 1 
h_dim = 128 #hidden 
lr = 0.00005 #learning_rate

# read data
df = pd.read_csv("../dataset/UGR/trainbot.csv")
print('original data label features number:')
print(df['Label'].value_counts())

# data preparation
criteria0 = df['Label'] == 1 #filter attack data 
attack_data = data0 = df[criteria0]

#float_array = data0.values.astype(float) 
#scaler = MinMaxScaler()
#scaled_array = scaler.fit_transform(float_array)
#attack_data = pd.DataFrame(scaled_array)
#attack_data.columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port','protocol',
 #               'flag1','flag2','flag3','flag4','flag5','flag6','fwd','stos','pkt','byt', 'Label']

print('----------------------------------------') 
print('attack value number:') 
print(attack_data['Label'].value_counts())

# transform data to tensor format
train_target = torch.tensor(attack_data['Label'].values.astype(np.float32)) 
train = torch.tensor(attack_data.drop(['Label'], axis = 1).values.astype(np.float32)) 
train_tensor = Data.TensorDataset(train, train_target) 

#creat DataLoader with batch and shuffle
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = mb_size, shuffle = True)

# define models
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

# Discriminator
D = torch.nn.Sequential( 
    torch.nn.Linear(X_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    # No sigmoid
    torch.nn.Linear(h_dim, 1),
)

# setup cpu using cpu (if use gpu -> set cpu to cuda)
G.cpu()
D.cpu()

def reset_grad():
    G.zero_grad()
    D.zero_grad()
    
#initialize optimizers and dataholders
G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
G_losses = []
D_losses = []
data_iter = iter(train_loader)

original data label features number:
0    14029
1     1430
Name: Label, dtype: int64
----------------------------------------
attack value number:
1    1430
Name: Label, dtype: int64


In [3]:
#training 
print("Start training WGAN model...")
d_iter = 2 
for it in range(500000):
    if it<100: d_iter = 50 
    else:
        d_iter = 2
    for _ in range(d_iter):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim)).cpu()
        try:
            X, _ = data_iter.next() 
        except StopIteration:
            data_iter = iter(train_loader) 
            X, _ = data_iter.next()
        #X = Variable(torch.from_numpy(X))
        X = X.cpu()
        
        # Dicriminator forward-loss-backward-update 
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        
        D_loss.backward() 
        D_solver.step()
        
        # Weight clipping 
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        D_losses.append(D_loss.data.cpu().numpy())
        
        # Housekeeping - reset gradient 
        reset_grad()
        
    # Generator forward-loss-backward-update 
    #X = Variable(torch.from_numpy(X)) 
    z = Variable(torch.randn(mb_size, z_dim)).cpu() #change cpu to cuda
    
    G_sample = G(z)
    D_fake = D(G_sample)
    
    G_loss = -torch.mean(D_fake)
    
    G_loss.backward() 
    G_solver.step()

    # Housekeeping - reset gradient 
    reset_grad()

    G_losses.append(G_loss.data.cpu().numpy())

    # Print and plot every now and then 
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.data.cpu().numpy(),  
                G_loss.data.cpu().numpy()))
        
        samples = G(z).data.numpy()[:16]

#plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Critic Loss During Training") 
plt.plot(G_losses,label="Generator") 
plt.plot(D_losses,label="Critic") 
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.savefig('UGR_loss_2.png')

#save model
torch.save({'modelG_state_dict': G.state_dict(), 
            'modelD_state_dict': D.state_dict(),
            'optimizerG_state_dict': G_solver.state_dict(),
            'optimizerD_state_dict': D_solver.state_dict(),
            'G_loss': G_losses,
            'D_loss': D_losses
           }, "WGAN_model_UGR_2.pth")

Start training WGAN model...
Iter-0; D_loss: -0.0013579567894339561; G_loss: -0.009504176676273346
Iter-1000; D_loss: -0.0018681702204048634; G_loss: -0.006461513228714466
Iter-2000; D_loss: -0.0004634074866771698; G_loss: -0.007668391801416874
Iter-3000; D_loss: -0.0012705451808869839; G_loss: -0.0070954859256744385
Iter-4000; D_loss: -0.0007117437198758125; G_loss: -0.008190728724002838
Iter-5000; D_loss: -0.000826861709356308; G_loss: -0.007653708104044199
Iter-6000; D_loss: -0.00027621258050203323; G_loss: -0.008402080275118351
Iter-7000; D_loss: -0.00019636843353509903; G_loss: -0.008807329460978508
Iter-8000; D_loss: -0.0003367215394973755; G_loss: -0.00908755511045456
Iter-9000; D_loss: -0.00014727097004652023; G_loss: -0.009047315455973148
Iter-10000; D_loss: -0.00018951017409563065; G_loss: -0.009166541509330273
Iter-11000; D_loss: -0.0001867702230811119; G_loss: -0.009185127913951874
Iter-12000; D_loss: -0.0002549579367041588; G_loss: -0.009530367329716682
Iter-13000; D_loss:

Iter-111000; D_loss: -2.113543450832367e-05; G_loss: -0.010873590596020222
Iter-112000; D_loss: -2.170167863368988e-05; G_loss: -0.011103563942015171
Iter-113000; D_loss: -0.0001428704708814621; G_loss: -0.011299291625618935
Iter-114000; D_loss: -3.8096681237220764e-05; G_loss: -0.011020716279745102
Iter-115000; D_loss: -7.393490523099899e-05; G_loss: -0.011017541401088238
Iter-116000; D_loss: -6.655789911746979e-05; G_loss: -0.011102650314569473
Iter-117000; D_loss: -5.512312054634094e-05; G_loss: -0.01119300164282322
Iter-118000; D_loss: -2.7443282306194305e-05; G_loss: -0.011195468716323376
Iter-119000; D_loss: 5.078501999378204e-06; G_loss: -0.011180234141647816
Iter-120000; D_loss: -3.873463720083237e-05; G_loss: -0.011016050353646278
Iter-121000; D_loss: -6.95548951625824e-05; G_loss: -0.011221716180443764
Iter-122000; D_loss: -0.00011205114424228668; G_loss: -0.011022930964827538
Iter-123000; D_loss: -8.598342537879944e-05; G_loss: -0.011343558318912983
Iter-124000; D_loss: -2.1

Iter-221000; D_loss: 2.7842819690704346e-05; G_loss: -0.013069015927612782
Iter-222000; D_loss: -8.011795580387115e-05; G_loss: -0.013318846002221107
Iter-223000; D_loss: 3.188382834196091e-05; G_loss: -0.013178622350096703
Iter-224000; D_loss: 5.291588604450226e-05; G_loss: -0.013185390271246433
Iter-225000; D_loss: 8.604861795902252e-05; G_loss: -0.013090710155665874
Iter-226000; D_loss: -1.9012950360774994e-05; G_loss: -0.013262508437037468
Iter-227000; D_loss: -6.3925981521606445e-06; G_loss: -0.012951239012181759
Iter-228000; D_loss: -2.8572045266628265e-05; G_loss: -0.012895002029836178
Iter-229000; D_loss: -2.9346905648708344e-05; G_loss: -0.013098673895001411
Iter-230000; D_loss: 9.161420166492462e-06; G_loss: -0.013107837177813053
Iter-231000; D_loss: -5.759671330451965e-05; G_loss: -0.01302725076675415
Iter-232000; D_loss: -2.2320076823234558e-05; G_loss: -0.01290790643543005
Iter-233000; D_loss: 1.1139549314975739e-05; G_loss: -0.012979463674128056
Iter-234000; D_loss: 4.690

Iter-331000; D_loss: -7.163733243942261e-06; G_loss: -0.013355942443013191
Iter-332000; D_loss: -1.6683712601661682e-05; G_loss: -0.013253524899482727
Iter-333000; D_loss: -1.4088116586208344e-05; G_loss: -0.013300606049597263
Iter-334000; D_loss: 1.9350089132785797e-05; G_loss: -0.013243751600384712
Iter-335000; D_loss: 6.717164069414139e-05; G_loss: -0.013376176357269287
Iter-336000; D_loss: -0.00012938398867845535; G_loss: -0.01358615793287754
Iter-337000; D_loss: -3.803800791501999e-05; G_loss: -0.013292774558067322
Iter-338000; D_loss: -4.770699888467789e-05; G_loss: -0.013320711441338062
Iter-339000; D_loss: 2.1248124539852142e-05; G_loss: -0.013204166665673256
Iter-340000; D_loss: 6.101094186306e-06; G_loss: -0.013172567822039127
Iter-341000; D_loss: -5.361437797546387e-05; G_loss: -0.013426244258880615
Iter-342000; D_loss: -3.2320618629455566e-05; G_loss: -0.013153813779354095
Iter-343000; D_loss: -2.226792275905609e-05; G_loss: -0.013456923887133598
Iter-344000; D_loss: 1.5405

Iter-441000; D_loss: -2.508796751499176e-05; G_loss: -0.012991715222597122
Iter-442000; D_loss: -5.418434739112854e-06; G_loss: -0.01304314099252224
Iter-443000; D_loss: -4.2164698243141174e-05; G_loss: -0.013227718882262707
Iter-444000; D_loss: -4.796311259269714e-07; G_loss: -0.013266418129205704
Iter-445000; D_loss: -2.7759000658988953e-05; G_loss: -0.013211198151111603
Iter-446000; D_loss: 9.96328890323639e-06; G_loss: -0.0132142324000597
Iter-447000; D_loss: -2.1428801119327545e-05; G_loss: -0.01293555274605751
Iter-448000; D_loss: -7.690303027629852e-05; G_loss: -0.013269073329865932
Iter-449000; D_loss: -5.725771188735962e-05; G_loss: -0.013194332830607891
Iter-450000; D_loss: 3.53250652551651e-05; G_loss: -0.013128329068422318
Iter-451000; D_loss: -0.00010762456804513931; G_loss: -0.013076134026050568
Iter-452000; D_loss: -3.7731602787971497e-05; G_loss: -0.013201948255300522
Iter-453000; D_loss: -3.8479454815387726e-05; G_loss: -0.013204798102378845
Iter-454000; D_loss: -0.000

# Data generation from saved WGAN model

In [4]:
#loaded NN model
print("Loading model...")
print("----------------------------------------")
json_file = open("DNN_critic_model_UGR_2.json","r") 
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json) 
loaded_model.load_weights("DNN_critic_model_UGR_2.h5")

G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

checkpoint = torch.load("WGAN_model_UGR_2.pth")
G.load_state_dict(checkpoint['modelG_state_dict'])

G.eval()
G.cpu()

final_Data = pd.DataFrame(columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port',
                                     'protocol','flag1','flag2','flag3','flag4','flag5','flag6',
                                     'fwd','stos','pkt','byt','Label']) 
print("empty df",final_Data) 
print("----------------------------------------")
print("Completed loading model")

Loading model...
----------------------------------------
empty df Empty DataFrame
Columns: [hour, minute, second, duration, source_ip, dest_ip, source_port, dest_port, protocol, flag1, flag2, flag3, flag4, flag5, flag6, fwd, stos, pkt, byt, Label]
Index: []
----------------------------------------
Completed loading model


In [5]:
it =0

while(final_Data.shape[0]<19000):
    it+=1
    z = Variable(torch.randn(100, z_dim)).cpu() 
    with torch.no_grad():
        data = G(z)
    data = data.cpu().numpy() 
    fake = pd.DataFrame(data)
        
    newy = np.ones((fake.shape[0],)) 
    #this is the model which changes 
    new_pred = loaded_model.predict(fake)
    new_pred = (new_pred > 0.5)
    cm = confusion_matrix(newy,new_pred)
    print("-------------------------------------------------------------") 
    print (" confusion_matrix for new data with deeper model \n" ,cm)
    
    combined_data = np.concatenate((fake,new_pred),axis=1) 
    combined_data = pd.DataFrame(combined_data)
    combined_data.columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port','protocol',
                'flag1','flag2','flag3','flag4','flag5','flag6','fwd','stos','pkt','byt', 'Label']
    
    criteria1 = combined_data['Label'] == 1 #just attack data 
    data1 = combined_data[criteria1]
    print("only attack data - ",data1.shape) 
    final_Data = final_Data.append(data1, ignore_index = True) 
    print("final data ", it, "- ",final_Data.shape) 
    print("-------------------------------------------------------------") 

print(final_Data['Label'].value_counts())
print("---------------------------------------------------------")
df2 = final_Data.sample(n=12599, replace=False, random_state=1)
df3 = final_Data.sample(n=19000, replace=False, random_state=1)
final = pd.concat([mydt,df2], ignore_index = True,axis=0)
final['Label'].value_counts()
df3.to_csv("../dataset/UGR/generated_bot.csv", index = False, header = True)
final.to_csv("../dataset/UGR/balanced_data_bot.csv", index = False, header = True)

4/4 [==============================] - 1s 4ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [34 66]]
only attack data -  (66, 20)
final data  1 -  (66, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step


/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [29 71]]
only attack data -  (71, 20)
final data  2 -  (137, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [39 61]]
only attack data -  (61, 20)
final data  3 -  (198, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [29 71]]
only attack data -  (71, 20)
final data  4 -  (269, 20)
-------------------------------------------------------------


/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [19 81]]
only attack data -  (81, 20)
final data  5 -  (350, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [25 75]]
only attack data -  (75, 20)
final data  6 -  (425, 20)
-------------------------------------------------------------


...

/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [21 79]]
only attack data -  (79, 20)
final data  256 -  (18879, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [21 79]]
only attack data -  (79, 20)
final data  257 -  (18958, 20)
-------------------------------------------------------------


/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [29 71]]
only attack data -  (71, 20)
final data  258 -  (19029, 20)
-------------------------------------------------------------
1.0    19029
Name: Label, dtype: int64
---------------------------------------------------------


/tmp/ipykernel_2630869/2681396701.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
